In [1]:
import numpy as np
from collections import Counter

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, mode='classification'):
        """
        Initialize Decision Tree.
        
        Parameters:
        - max_depth: Maximum depth of the tree (default: None, no limit)
        - min_samples_split: Minimum number of samples required to split a node (default: 2)
        - mode: 'classification' or 'regression' (default: 'classification')
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.mode = mode
        self.tree = None

    def fit(self, X, y):
        """
        Train the Decision Tree.
        
        Parameters:
        - X: Feature matrix (n_samples, n_features)
        - y: Target values (n_samples,)
        """
        self.tree = self._build_tree(X, y, depth=0)

    def predict(self, X):
        """
        Make predictions using the trained Decision Tree.
        
        Parameters:
        - X: Feature matrix (n_samples, n_features)
        
        Returns:
        - Predictions (n_samples,)
        """
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _build_tree(self, X, y, depth):
        """
        Recursively build the Decision Tree.
        """
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions
        if (self.max_depth is not None and depth >= self.max_depth) or \
           n_samples < self.min_samples_split or \
           n_labels == 1:
            return self._create_leaf(y)

        # Find the best split
        best_split = self._find_best_split(X, y, n_samples, n_features)
        if not best_split:
            return self._create_leaf(y)

        # Split the data
        left_idxs, right_idxs = best_split['left_idxs'], best_split['right_idxs']
        left = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return {
            'feature_index': best_split['feature_index'],
            'threshold': best_split['threshold'],
            'left': left,
            'right': right
        }

    def _find_best_split(self, X, y, n_samples, n_features):
        """
        Find the best split for the data.
        """
        best_split = {}
        best_gain = -float('inf')

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_idxs = np.where(X[:, feature_index] <= threshold)[0]
                right_idxs = np.where(X[:, feature_index] > threshold)[0]

                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                # Calculate information gain or variance reduction
                if self.mode == 'classification':
                    gain = self._information_gain(y, y[left_idxs], y[right_idxs])
                elif self.mode == 'regression':
                    gain = self._variance_reduction(y, y[left_idxs], y[right_idxs])

                if gain > best_gain:
                    best_gain = gain
                    best_split = {
                        'feature_index': feature_index,
                        'threshold': threshold,
                        'left_idxs': left_idxs,
                        'right_idxs': right_idxs,
                        'gain': gain
                    }

        return best_split

    def _information_gain(self, y, y_left, y_right):
        """
        Calculate information gain for classification.
        """
        p = len(y_left) / len(y)
        return self._entropy(y) - p * self._entropy(y_left) - (1 - p) * self._entropy(y_right)

    def _entropy(self, y):
        """
        Calculate entropy for classification.
        """
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

    def _variance_reduction(self, y, y_left, y_right):
        """
        Calculate variance reduction for regression.
        """
        p = len(y_left) / len(y)
        return np.var(y) - p * np.var(y_left) - (1 - p) * np.var(y_right)

    def _create_leaf(self, y):
        """
        Create a leaf node.
        """
        if self.mode == 'classification':
            return Counter(y).most_common(1)[0][0]  # Majority class
        elif self.mode == 'regression':
            return np.mean(y)  # Mean value

    def _traverse_tree(self, x, node):
        """
        Traverse the tree to make a prediction.
        """
        if isinstance(node, dict):
            feature_index = node['feature_index']
            threshold = node['threshold']
            if x[feature_index] <= threshold:
                return self._traverse_tree(x, node['left'])
            else:
                return self._traverse_tree(x, node['right'])
        else:
            return node  # Leaf node


In [2]:
# Example usage
if __name__ == "__main__":
    # Classification example
    print("Decision Tree Classification Example")
    print("===================================")
    X = np.array([[2, 3], [10, 15], [3, 4], [6, 8], [7, 10], [8, 12]])
    y = np.array([0, 1, 0, 1, 1, 1])

    clf = DecisionTree(max_depth=3, mode='classification')
    clf.fit(X, y)
    predictions = clf.predict(X)
    print(f"Predictions: {predictions}")
    print(f"Accuracy: {np.mean(predictions == y):.2f}")

    # Regression example
    print("\nDecision Tree Regression Example")
    print("================================")
    X = np.array([[1], [2], [3], [4], [5], [6]])
    y = np.array([2, 4, 6, 8, 10, 12])

    reg = DecisionTree(max_depth=3, mode='regression')
    reg.fit(X, y)
    predictions = reg.predict(X)
    print(f"Predictions: {predictions}")
    print(f"MSE: {np.mean((predictions - y) ** 2):.2f}")

Decision Tree Classification Example
Predictions: [0 1 0 1 1 1]
Accuracy: 1.00

Decision Tree Regression Example
Predictions: [ 2.  4.  6.  8. 10. 12.]
MSE: 0.00
